<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine%20v5.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
# requirements
import nltk
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import requests
import operator
import flask
from keras.preprocessing.text import Tokenizer
import re
import string
from wordcloud import STOPWORDS
! pip install pyspellchecker
from spellchecker import SpellChecker
from gensim.models.keyedvectors import KeyedVectors
import warnings
warnings.filterwarnings('ignore')


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
CD.npy	ICD_dict.npy  map_abbreviation.npy  wikipedia-pubmed-and-PMC-w2v.bin


In [0]:
# Building vectors for CD
prohibitedWords = ['other','unspecified','cause','[any part, except buttock]','[any part, except face]','not elsewhere classified','part','parts','specific','region','site','sites','specified','Unspecified','Generalized','(s)','multiple','due to medical condition','type','initial encounter','subsequent encounter']
big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))

def onetimeclean(text):
  x = wpt.tokenize(big_regex.sub("", text.lower()))
  x = [w for w in x if w not in list(STOPWORDS)+['(',')','[',']','-',"'",':',',','.','{','}','_']]
  vect = np.array([np.zeros((200,)) if word not in model.wv.vocab else model.wv[word] for word in x])
  if vect.shape[0]<11:
    vect = np.vstack((vect,np.zeros((11-vect.shape[0],200))))
  if vect.shape != (11,200): print("vector shape not consistent: {}".format(text))
  return vect

#CD = np.array([onetimeclean(code) for code in dict(ICD_dict).values()])
#np.save('/content/drive/My Drive/CCP-ICDsearch/CD.npy', CD)

# Functions
def clean(text):
  x = wpt.tokenize(text.lower())
  x = [w for w in x if w not in list(STOPWORDS)+['(',')','-',"'",':',',','.','{','}','_']]
  x = [map_abbreviation[word] if word in list(map_abbreviation.keys()) else word for word in x]
  x = list(set([spell.correction(word) if word not in model.wv.vocab else word for word in x ]))
  vect = np.array([np.zeros((200,)) if word not in model.wv.vocab else model.wv[word] for word in x])
  return vect

In [0]:
# Initializers
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
wpt = nltk.WordPunctTokenizer()
spell = SpellChecker()

In [0]:
# KB
ICD_dict = np.load('/content/drive/My Drive/CCP-ICDsearch/ICD_dict.npy')
ICD_dict = sorted(dict(ICD_dict).items(), key=lambda kv: kv[0])
map_abbreviation = np.load('/content/drive/My Drive/CCP-ICDsearch/map_abbreviation.npy').item()
#Build/process and save CD (n_icd=2185, fix_len_cd=11, emb_size=200): 
CD=np.load('/content/drive/My Drive/CCP-ICDsearch/CD.npy')


In [146]:
search_term = "cardiac arrest"

DD = clean(search_term)
n = np.sum(np.dot(CD,DD.T),axis=(1,2))
d1 = sum(np.diagonal(np.dot(DD,DD.T)))
d2 = np.count_nonzero(np.sum(CD, axis=2), axis=1) + 0.00001
out = n/(d1*d2)
index = np.argsort(out)[-15:]
scores = out[index[::-1]]
result = np.array(ICD_dict)[index[::-1]]
df = pd.DataFrame(result,scores*100)
df.reset_index(level=0, inplace=True)
df.columns = ['Confidence','ICD-10','Description']
df

,Confidence,ICD-10,Description
0,71.138845,I469,"Cardiac arrest, cause unspecified"
1,60.798841,I498,Other specified cardiac arrhythmias
2,58.360368,I4901,Ventricular fibrillation
3,57.773877,I472,Ventricular tachycardia
4,57.311926,I499,"Cardiac arrhythmia, unspecified"
5,56.683625,I471,Supraventricular tachycardia
6,54.250983,I4902,Ventricular flutter
7,51.609104,I4891,Unspecified atrial fibrillation
8,47.920810,I97710,Intraoperative cardiac arrest during cardiac surgery
9,47.499719,I4892,Unspecified atrial flutter
